In [1]:
#!pip install earthengine-api
#!pip install geehydro
#!pip install geemap
#!pip install geetools

In [2]:
import ee
import folium
import geemap
import geetools
import os

In [3]:
# Authenticate & Initializing
#ee.Authenticate()
ee.Initialize()

### NDVI 
NDVI is a proxy for the plat productivity or a measure of the state of plant health.
​
wavelengths: nir - 800nmred - 670nm
formula: NDVI = (nir - red) / (nir + red)

In [4]:
def ndviFun_sent(im):
  ndvi = im.normalizedDifference(['B8','B4']).rename('ndvi')
  return im.addBands(ndvi)

### NDWI

NDWI returns normalized difference water index. It monitor changes in water content of leaves.

Formula: NIR - 860nm, SWIR - 1240nm
NDWI = (NIR - SWIR) / (NIR + SWIR)

In [5]:
def ndwiFun_sent(im):
  ndwi = im.normalizedDifference(['B4','B11']).rename('ndwi')
  return im.addBands(ndwi)

### RGRI 
The Red Green Ratio Index is a measure of the ratio between the
reflectance in the Red Band and the Green Band.
This index is a reflectance measurement that indicates the relative
expression of leaf redness causedb y anthocyanin to that of chlorophyll.
The Red Green Ratio has been used to estimate the course of foliage
development in canopies. It is an indicator of leaf production and stress,
and it may also indicate flowering in some canopies. Applications include
plant growth cycle (phenology) studies, canopy stress detection, and crop
yield prediction. Results are reported as the mean of all bands in the red
range divided by the mean of all bands in the green range.

In [6]:
def rgriFun_sent(im):
    green = im.select('B3') # get green band
    red = im.select('B4') # get red band
    rgri = red.divide(green).rename('rgri')
    return im.addBands(rgri)

In [7]:
shp = ee.FeatureCollection('users/ta346/tompkins-ag-district')

In [8]:
from geetools import cloud_mask
mask_LS_SR = cloud_mask.sentinel2()

sent = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(shp) \
    .filter(ee.Filter.dayOfYear(150, 240)) \
    .filterDate('2020-05-01', '2020-08-31') \
    .map(mask_LS_SR) \
    .map(ndwiFun_sent) \
    .map(ndviFun_sent) \
    .map(rgriFun_sent)

rgri = sent.select("rgri")
ndvi = sent.select("ndvi")
ndwi = sent.select("ndwi")

output_rgri = rgri.reduce(ee.Reducer.mean()).clip(shp)
output_ndvi = ndvi.reduce(ee.Reducer.mean()).clip(shp)
output_ndwi = ndwi.reduce(ee.Reducer.mean()).clip(shp) 

In [9]:
rgri = ["9D6539","B17442","C5834B",
        "D99255","B5A854","92BE53",
        "6FD452","6DC84F","6CBC4D",
        "6BB04B","6AA449","699847", "688C45"]

def reverse_l(lst):
    lst.reverse() 
    return lst

# rgri color function to visualize later
colorized_rgri = {
    'min': 0.0,
    'max': 1.0,
    'palette': reverse_l(rgri)
}

# ndvi color function to visualize later
colorized_ndvi = {
  'min': 0.3,
  'max': 1.0,
  'palette': ["FFFFFF","CE7E45","DF923D",
        'F1B555','FCD163','99B718',
        '74A901','66A000','529400',
        '3E8601','207401','056201',
        '004C00', '023B01','012E01',
        '011D01', '011301']
}

# ndwi color function to visualize later
colorized_ndwi = {
  'min': -1.0,
  'max': 1.0,
  'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
}

In [10]:
Map_2 = geemap.Map()
Map_2.add_basemap(basemap = "SATELLITE")
#Map_2.basemap_demo()
Map_2.centerObject(shp, zoom=10)

In [11]:
Map_2.addLayer(shp, {}, "tompkins-ag-district", opacity = 0.7)

In [12]:
Map_2.addLayer(output_rgri, colorized_rgri, "sentinal_rgri")
Map_2.addLayer(output_ndvi, colorized_ndvi, "sentinal_ndvi")
Map_2.addLayer(output_ndwi, colorized_ndwi, "seninal_ndwi")

In [13]:
#pip install virtualenv

In [14]:
!virtualenv sendsms

created virtual environment CPython3.9.1.final.0-64 in 6663ms
  creator CPython3Windows(dest=C:\Users\khuse\Dropbox\Digital Hackathon\sendsms, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\khuse\AppData\Local\pypa\virtualenv)
    added seed packages: pip==21.0.1, setuptools==52.0.0, setuptools==53.0.0, wheel==0.36.2
  activators BashActivator,BatchActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [15]:
!activate


(gee) C:\Users\khuse\Dropbox\Digital Hackathon>conda.bat activate  


In [16]:
#pip install twilio>=6.0.0

In [17]:
Map_2

Map(center=[42.4734929983418, -76.47636961932672], controls=(WidgetControl(options=['position', 'transparent_b…

In [18]:
from twilio.rest import Client 
client = Client("AC868e3ba4b6bb8ecd11c50f69961c7b79","b0f8f790262e8dc1a41a0b1a098a6da3")
client.messages.create(to="+13157412495",from_="+17866500711", body="Farm eTech : Plot 1 shows the low growth of plant productivity as NDVI has gone down by 4.2% as compared to that of last month")

<Twilio.Api.V2010.MessageInstance account_sid=AC868e3ba4b6bb8ecd11c50f69961c7b79 sid=SM633081c788534d74936d30d3d0125f5f>